In [55]:
import numpy as np
from deepsy.core.model import Model
from deepsy.core.nn import NeuralNetwork
from deepsy.core.layer import Layer
from deepsy.core.functions.activation_functions import *
from deepsy.core.functions.loss_functions import *
from deepsy.core.functions.optimizers import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [56]:
dataset = pd.read_csv('data/binary/titanic.csv')
Y = dataset['Survived']
X = dataset.drop(columns=['Survived'])
X = X.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
X = pd.get_dummies(X, columns=['Pclass', 'Sex', 'Embarked'])

X['Age'].fillna(X['Age'].mean(), inplace=True)
X['Fare'].fillna(X['Fare'].mean(), inplace=True)

X = (X - X.mean()) / X.std()

X, Y = X.values, Y.values

print('Dataset shape: {}'.format(dataset.shape))
print(dataset.head())
print()
print(X[:5])
print()
print(Y[:5])
print()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

nn = NeuralNetwork(layers=[
    Layer(nr_neurons=12, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=16, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=8, nr_in_features=16, activation_func=ReLU()),
    Layer(nr_neurons=1, nr_in_features=8, activation_func=Sigmoid())
])

model = Model(nn, loss_func=BinaryCrossEntropy(), optimizer=GradientDescent(learning_rate=0.1))
model.train(X_train.T, Y_train.T, 400)

Dataset shape: (891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8

In [57]:
Y_predicted = model.predict(X_test.T)
acc = sum((Y_predicted.reshape(Y_test.shape) > 0.5) == Y_test) / len(Y_test)
print('Accuracy: {}'.format(acc))
for i in range(len(Y_predicted[0])):
    print('predicted: {}   actual: {}'.format(Y_predicted[0][i], Y_test[i]))

Accuracy: 0.8100558659217877
predicted: 0.9908497283109211   actual: 1
predicted: 0.007409758840963713   actual: 1
predicted: 0.00010626383916587735   actual: 1
predicted: 0.7943937578879157   actual: 0
predicted: 0.08062138807698958   actual: 0
predicted: 0.9993931696832671   actual: 1
predicted: 0.14462077027777084   actual: 0
predicted: 0.14933375964555745   actual: 0
predicted: 0.14462077027777084   actual: 0
predicted: 0.9999620563069302   actual: 1
predicted: 0.013058715853320068   actual: 0
predicted: 0.10385689174511553   actual: 0
predicted: 0.09212816460133183   actual: 0
predicted: 0.0006193338334121135   actual: 1
predicted: 3.208689128629945e-05   actual: 1
predicted: 0.00018078009377536894   actual: 0
predicted: 0.0024324397644950265   actual: 1
predicted: 0.9995834544209873   actual: 1
predicted: 0.1297281856046262   actual: 0
predicted: 0.9972327821467644   actual: 0
predicted: 0.002224489041585368   actual: 0
predicted: 0.14451671917588382   actual: 0
predicted: 0.9999

In [58]:
dataset = pd.read_csv('data/binary/Employee.csv')
Y = dataset['LeaveOrNot']
X = dataset.drop(columns=['LeaveOrNot'])
X = pd.get_dummies(X, columns=['Education', 'City', 'Gender', 'EverBenched'])

X = (X - X.mean()) / X.std()

X, Y = X.values, Y.values

print('Dataset shape: {}'.format(dataset.shape))
print(dataset.head())
print()
print(X[:5])
print()
print(Y[:5])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

nn = NeuralNetwork(layers=[
    Layer(nr_neurons=12, nr_in_features=14, activation_func=ReLU()),
    Layer(nr_neurons=16, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=8, nr_in_features=16, activation_func=ReLU()),
    Layer(nr_neurons=1, nr_in_features=8, activation_func=Sigmoid())
])

model = Model(nn, loss_func=BinaryCrossEntropy(), optimizer=GradientDescent(1))
model.train(X_train.T, Y_train.T, 500)

Dataset shape: (4653, 9)
   Education  JoiningYear       City  PaymentTier  Age  Gender EverBenched  \
0  Bachelors         2017  Bangalore            3   34    Male          No   
1  Bachelors         2013       Pune            1   28  Female          No   
2  Bachelors         2014  New Delhi            3   38  Female          No   
3    Masters         2016  Bangalore            3   27    Male          No   
4    Masters         2017       Pune            3   24    Male         Yes   

   ExperienceInCurrentDomain  LeaveOrNot  
0                          0           0  
1                          3           1  
2                          2           0  
3                          5           1  
4                          2           1  

[[ 1.03952665  0.53744523  0.95454254 -1.86470098  0.54044268 -0.48052341
  -0.20000086  1.04316163 -0.57522042 -0.61197491 -0.82146269  0.82146269
   0.33832885 -0.33832885]
 [-1.10711376 -3.024852   -0.28870069  0.06054761  0.54044268 -0.4805234

In [59]:
Y_predicted = model.predict(X_test.T)
acc = sum((Y_predicted.reshape(Y_test.shape) > 0.5) == Y_test) / len(Y_test)
print('Accuracy: {}'.format(acc))
for i in range(len(Y_predicted[0])):
    print('predicted: {}   actual: {}'.format(Y_predicted[0][i], Y_test[i]))

Accuracy: 0.6949516648764769
predicted: 1.2755014397259982e-10   actual: 0
predicted: 0.9997879733318192   actual: 1
predicted: 0.959058681321088   actual: 1
predicted: 4.430671277667477e-06   actual: 0
predicted: 5.374371018947509e-06   actual: 0
predicted: 0.1650472355627405   actual: 0
predicted: 2.0485343153029834e-08   actual: 0
predicted: 0.00035442811793045116   actual: 0
predicted: 0.9999987321281997   actual: 1
predicted: 0.021873105775450378   actual: 1
predicted: 1.1042498086595548e-18   actual: 0
predicted: 3.8628970100023e-05   actual: 1
predicted: 5.931314569980685e-13   actual: 0
predicted: 6.100183411957048e-07   actual: 0
predicted: 2.443381797496994e-18   actual: 0
predicted: 3.224514688231572e-14   actual: 1
predicted: 0.9999587655870417   actual: 1
predicted: 1.38137598325347e-19   actual: 0
predicted: 8.661311842143913e-11   actual: 0
predicted: 0.9999987866006108   actual: 1
predicted: 8.661311842143913e-11   actual: 0
predicted: 0.979822707701414   actual: 0
pred

In [60]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, (Y_predicted.reshape(Y_test.shape) > 0.5)))

              precision    recall  f1-score   support

           0       0.74      0.81      0.77       600
           1       0.58      0.49      0.53       331

    accuracy                           0.69       931
   macro avg       0.66      0.65      0.65       931
weighted avg       0.69      0.69      0.69       931

